# CNN Model

In [1]:
# Import the necessary libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Set up the data generators for training and validation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load the image data
image_dir = './FMD/image/'
class_names = os.listdir(image_dir)

X = []
y = []

for i, class_name in enumerate(class_names):
    class_dir = os.path.join(image_dir, class_name)
    for image_path in os.listdir(class_dir):
        image = keras.preprocessing.image.load_img(os.path.join(class_dir, image_path), target_size=(224, 224))
        image_array = keras.preprocessing.image.img_to_array(image)
        X.append(image_array)
        y.append(i)

In [3]:
# Convert the lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Perform reshaping operation
#X_train = X_train.reshape(X_train.shape[0],28,28,1)
#X_test = X_test.reshape(X_test.shape[0],28,28,1)

In [8]:
# Normalization
X_train = X_train/255
X_test = X_test/255

In [9]:
# One-Hot Encoding
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [10]:
# Define the CNN model architecture
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [11]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=128,
    verbose=1,
    validation_data=(X_test, y_test)
)

Epoch 1/20
7/7 [==============================] - 28s 4s/step - loss: 4.0389 - accuracy: 0.1000 - val_loss: 2.2648 - val_accuracy: 0.1500
Epoch 2/20
7/7 [==============================] - 29s 4s/step - loss: 2.2342 - accuracy: 0.1713 - val_loss: 2.3522 - val_accuracy: 0.1150
Epoch 3/20
7/7 [==============================] - 28s 4s/step - loss: 2.2011 - accuracy: 0.2025 - val_loss: 2.1639 - val_accuracy: 0.1450
Epoch 4/20
7/7 [==============================] - 28s 4s/step - loss: 2.1108 - accuracy: 0.2150 - val_loss: 2.0752 - val_accuracy: 0.2750
Epoch 5/20
7/7 [==============================] - 28s 4s/step - loss: 2.0617 - accuracy: 0.2788 - val_loss: 2.0648 - val_accuracy: 0.2700
Epoch 6/20
7/7 [==============================] - 28s 4s/step - loss: 2.0111 - accuracy: 0.2912 - val_loss: 2.1206 - val_accuracy: 0.1900
Epoch 7/20
7/7 [==============================] - 28s 4s/step - loss: 1.9921 - accuracy: 0.2775 - val_loss: 2.0109 - val_accuracy: 0.2900
Epoch 8/20
7/7 [==================

In [14]:
# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=2)
('Test Loss:', score[0])
('Test accuracy:', score[1])

7/7 - 2s - loss: 6.3029 - accuracy: 0.2300 - 2s/epoch - 234ms/step


('Test accuracy:', 0.23000000417232513)

# LeNet 5 Model

In [15]:
# Define the LeNet 5 model architecture
model = keras.Sequential([
    layers.Conv2D(6,kernel_size=(5,5),activation='relu',input_shape=(224,224,3)),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(16, kernel_size=(5,5), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    layers.Dense(84, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=128,
    verbose=1,
    validation_data=(X_test, y_test)
)

# Evaluate the model on the testing set
score = model.evaluate(X_test, y_test, verbose=2)
('Test Loss:', score[0])
('Test accuracy:', score[1])

Epoch 1/20
7/7 [==============================] - 13s 2s/step - loss: 3.2399 - accuracy: 0.1238 - val_loss: 2.3980 - val_accuracy: 0.0900
Epoch 2/20
7/7 [==============================] - 11s 2s/step - loss: 2.2954 - accuracy: 0.1150 - val_loss: 2.2691 - val_accuracy: 0.1500
Epoch 3/20
7/7 [==============================] - 12s 2s/step - loss: 2.2219 - accuracy: 0.2188 - val_loss: 2.2679 - val_accuracy: 0.1900
Epoch 4/20
7/7 [==============================] - 12s 2s/step - loss: 2.1529 - accuracy: 0.2925 - val_loss: 2.2614 - val_accuracy: 0.1350
Epoch 5/20
7/7 [==============================] - 13s 2s/step - loss: 2.0453 - accuracy: 0.2862 - val_loss: 2.1993 - val_accuracy: 0.2100
Epoch 6/20
7/7 [==============================] - 15s 2s/step - loss: 1.8794 - accuracy: 0.4038 - val_loss: 2.1601 - val_accuracy: 0.2200
Epoch 7/20
7/7 [==============================] - 14s 2s/step - loss: 1.6838 - accuracy: 0.5013 - val_loss: 2.1638 - val_accuracy: 0.2500
Epoch 8/20
7/7 [==================

('Test accuracy:', 0.2750000059604645)